# Price Scraper for RugsUSA

In [406]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
from random import randint
from time import sleep

In [407]:
os.getcwd()

'/Users/Daniel.Kwon/projects/web-scraper'

In [408]:
def extract_source(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
    source=requests.get(url, headers=headers)
    return source

In [409]:
#first_page = soup.find(id='ui_target')
#print(first_page.prettify())
#last_page_elem = first_page.find('span',class_="suffix")
#last_page = last_page_elem.text
#last_page = [int(s) for s in last_page.split() if s.isdigit()][0]
#if last_page > 10: 
#    max_page = 5
#else:
#    max_page = last_page

In [410]:
df_scraped_prices = None

In [411]:
URL = "https://www.rugsusa.com/rugsusa/control/search-rugs"
#URL = "https://www.rugsusa.com/rugsusa/control/search-rugs?style_group=Country%20%26%20Floral"
page = extract_source(URL)
soup = BeautifulSoup(page.text, 'html.parser')

In [412]:
filters = soup.find(id='facetContainer')
size_filters = filters.find_all('a',{"class": "facetcouple",
                                     "data-filter": "size_grp"})
style_filters = filters.find_all('a',{"class": "facetcouple",
                                     "data-filter": "style_group"})

In [413]:
data = []
for style_filter in style_filters:
    style_option = style_filter.find('span',class_='facet_val')
    for size_filter in size_filters:
        size_option = size_filter.find('span',class_='facet_val')
        new_URL = "https://www.rugsusa.com/rugsusa/control/search-rugs?style_group={style}&size_grp={size}".format(style=style_option.text.
                                                                                                                   replace(' & ','%20%26%20').
                                                                                                                   replace(' ','%20'),
                                                                                                                   size=size_option.text.
                                                                                                                   replace(' ','%20'))
        new_page = extract_source(new_URL)
        sleep(randint(1,5))
        new_soup = BeautifulSoup(new_page.text, 'html.parser')
        results = new_soup.find(id='searchTarget')
        rug_listings = results.find_all('div',class_="resultTile prod productCard productinfoimagewithbadgesmask")
        for rug_listing in rug_listings:
            name_elem = rug_listing.find('div',class_='productname1 head-text')
            msrp_price_elem = rug_listing.find('span', class_='msrp body-text')
            default_price_elem = rug_listing.find('span', class_='default_price accent-text')
            num_review_elem = rug_listing.find('div', class_ = 'amountofreviews')
            image_url = rug_listing.find('div', class_= 'imagewithbadges')['data-secondary-img-url']
            data.append([name_elem.text, 
                         'N/A' if msrp_price_elem is None else msrp_price_elem.text, 
                         'N/A' if default_price_elem is None else default_price_elem.text,
                         -1 if num_review_elem is None else int(num_review_elem.text.replace('','').replace('(','').replace(')','')),
                         size_option.text,
                         style_option.text,
                         'N/A' if image_url is None else image_url
                        ])

In [416]:
if df_scraped_prices is None:
    df_scraped_prices = pd.DataFrame(data, columns=["rug_name","msrp_price","list_price","num_reviews","size","category","image_url"])
else:
    df_data = pd.DataFrame(data, columns=["rug_name","msrp_price","list_price","num_reviews","size","category","image_url"])
    df_scraped_prices = df_scraped_prices.append(df_data)
df_scraped_prices.drop_duplicates(inplace=True)

In [417]:
df_scraped_prices

,rug_name,msrp_price,list_price,num_reviews,size,category,image_url
0,2' x 3' Leopard Print Rug,$60,$26,417,2x3,Animal Prints,//www.rug-images.com/products/s/200RZBD61A.jpg...
1,2' x 3' Contemporary Zebra Print with Faux Sil...,$100,$39,415,2x3,Animal Prints,//www.rug-images.com/products/s/200ZF5.jpg?pur...
2,2' x 3' ATH5120 Rug,N/A,N/A,2,2x3,Animal Prints,//www.rug-images.com/products/s/158ATH5120.jpg...
3,2' x 3' ATH5120 Rug,N/A,N/A,2,2x3,Animal Prints,//www.rug-images.com/products/s/158ATH5000.jpg...
4,2' x 3' ATH5120 Rug,N/A,N/A,2,2x3,Animal Prints,//www.rug-images.com/products/s/158ATH5114.jpg...
...,...,...,...,...,...,...,...
15313,"2' 6"" x 10' Faded Abstract Washable Rug",$220,$99,10,10ft and up Runner,Washable,//www.rug-images.com/products/s/200BIRV08A.jpg...
15314,"2' 6"" x 10' Rosette Medallion Washable Rug",$220,$99,18,10ft and up Runner,Washable,//www.rug-images.com/products/s/200BIRV02A.jpg...
15315,"2' 6"" x 10' Hazy Heraldry Washable Rug",$220,$99,17,10ft and up Runner,Washable,//www.rug-images.com/products/s/200HJAU13A.jpg...
15316,"2' 6"" x 10' Jungle Washable Rug",$220,$99,5,10ft and up Runner,Washable,//www.rug-images.com/products/s/200HJAU01A.jpg...


In [418]:
df_scraped_prices.to_csv('{}/rugsUSA_scraped_prices.csv'.format(os.getcwd()), index=False)

### Competitors
- 6 comepttiors
- UK competitors / comps